## Part 1: Identify providers that overcharge for certain procedures or regions where procedures are too expensive.
    


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import pyodbc

In [2]:
# Step 1: Connect to SQL Server
# Define connection parameters
server = 'DESKTOP-DGDUOBT\SQLEXPRESS'  
database = 'MedicareAnalysis'
username = ''  
password = ''  
driver = '{ODBC Driver 17 for SQL Server}'   

try:
    conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};Trusted_Connection=yes')
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")

Connection successful!


In [3]:
# Step 2: Load data from SQL tables
Medicare_Charge_In = pd.read_sql_query("SELECT * FROM Medicare_Charge_Inpatient_DRG", conn)
Medicare_Charge_Out = pd.read_sql_query("SELECT * FROM Medicare_Charge_Outpatient_APC", conn)
Medicare_Provider_Charge_In = pd.read_sql_query("SELECT * FROM Provider_Charge_Inpatient_DRG", conn)
Medicare_Provider_Charge_Out = pd.read_sql_query("SELECT * FROM Provider_Charge_Outpatient_APC", conn)


# Step 3: Close the connection after loading data
conn.close()
print("Data loaded successfully & connection closed.")

C:\Users\HP\AppData\Local\Temp\ipykernel_9084\1251983331.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Medicare_Charge_In = pd.read_sql_query("SELECT * FROM Medicare_Charge_Inpatient_DRG", conn)
C:\Users\HP\AppData\Local\Temp\ipykernel_9084\1251983331.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Medicare_Charge_Out = pd.read_sql_query("SELECT * FROM Medicare_Charge_Outpatient_APC", conn)
C:\Users\HP\AppData\Local\Temp\ipykernel_9084\1251983331.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Medicare_Provide

Data loaded successfully & connection closed.


In [6]:
Medicare_Provider_Charge_In.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163065 entries, 0 to 163064
Data columns (total 12 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   DRG_Definition             163065 non-null  object 
 1   Provider_Id                163065 non-null  int64  
 2   Provider_Name              163065 non-null  object 
 3   Provider_Street_Address    163065 non-null  object 
 4   Provider_City              163065 non-null  object 
 5   Provider_State             163065 non-null  object 
 6   Provider_Zip_Code          163065 non-null  int64  
 7   Hospital_Referral_Region   163065 non-null  object 
 8   Total_Discharges           163065 non-null  int64  
 9   Average_Covered_Charges    163065 non-null  float64
 10  Average_Total_Payments     163065 non-null  float64
 11  Average_Medicare_Payments  163065 non-null  float64
dtypes: float64(3), int64(3), object(6)
memory usage: 14.9+ MB


In [7]:
Medicare_Provider_Charge_Out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43372 entries, 0 to 43371
Data columns (total 11 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   APC                                       43372 non-null  object 
 1   Provider_Id                               43372 non-null  int64  
 2   Provider_Name                             43372 non-null  object 
 3   Provider_Street_Address                   43372 non-null  object 
 4   Provider_City                             43372 non-null  object 
 5   Provider_State                            43372 non-null  object 
 6   Provider_Zip_Code                         43372 non-null  int64  
 7   Hospital_Referral_Region_HRR_Description  43372 non-null  object 
 8   Outpatient_Services                       43372 non-null  int64  
 9   Average_Estimated_Submitted_Charges       43372 non-null  float64
 10  Average_Total_Payments            

## Part 1A: Highest Cost Variation

### INPATIENT Treatment

We'll analyze cost variation based on "Average_Covered_Charges" (amount providers charge Medicare) for each DRG Diagnosis.

In [4]:
# Group by procedure and calculate min, max, and standard deviation of charges
cost_variation = Medicare_Provider_Charge_In.groupby('DRG_Definition')['Average_Covered_Charges'].agg(['min', 'max', 'mean', 'std'])
cost_variation.head(2)

,min,max,mean,std
DRG_Definition,,,,
039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,5981.053711,146892.0000,30481.989332,15941.252051
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,5617.937500,148128.4375,24307.762692,14085.285701


#### **Method 1: Absolute Variation (Max - Min) / Mean**

In [89]:
cost_variation['Range_Variation_Percentage'] = ((cost_variation['max'] - cost_variation['min']) / cost_variation['mean']) * 100

# Sort by highest variation (absolute method)
cost_variation_sorted = cost_variation.sort_values(by='Range_Variation_Percentage', ascending=False)
cost_variation_sorted.head(5)

,min,max,mean,std,Range_Variation_Percentage
DRG Definition,,,,,
812 - RED BLOOD CELL DISORDERS W/O MCC,3534.000000,166362.5217,19723.333778,11059.765733,825.562877
885 - PSYCHOSES,2749.942857,144522.6667,19929.455817,14305.274994,711.372780
207 - RESPIRATORY SYSTEM DIAGNOSIS W VENTILATOR SUPPORT 96+ HOURS,29636.636360,929118.9091,143428.055060,84817.423101,627.131332
389 - G.I. OBSTRUCTION W CC,5007.636364,151705.5294,24565.592240,13751.815749,597.168151
057 - DEGENERATIVE NERVOUS SYSTEM DISORDERS W/O MCC,5617.937500,148128.4375,24307.762697,14085.285662,586.275676


#### **Method 2: Relative Variation (Standard Deviation / Mean) - Coefficient of Variation**

In [5]:
cost_variation['CV_Percentage'] = (cost_variation['std'] / cost_variation['mean']) * 100

# Sort by highest variation (Relative method)
cost_variation_sorted = cost_variation.sort_values(by='CV_Percentage', ascending=False)
cost_variation_sorted.head(5)

,min,max,mean,std,CV_Percentage
DRG_Definition,,,,,
885 - PSYCHOSES,2749.942871,144522.671875,19929.455790,14305.275033,71.779557
897 - ALCOHOL/DRUG ABUSE OR DEPENDENCE W/O REHABILITATION THERAPY W/O MCC,2536.699219,79932.539062,17589.098716,11150.914643,63.396737
917 - POISONING & TOXIC EFFECTS OF DRUGS W MCC,6809.833496,236455.906250,40345.535841,24737.866921,61.315004
190 - CHRONIC OBSTRUCTIVE PULMONARY DISEASE W MCC,4744.461426,140771.703125,29140.948466,17786.867929,61.037368
189 - PULMONARY EDEMA & RESPIRATORY FAILURE,7199.866699,169831.953125,34276.195572,20912.138154,61.010675


#### OUTPATIENT Treatment

In [11]:
cost_variation_out = Medicare_Provider_Charge_Out.groupby('APC')['Average_Estimated_Submitted_Charges'].agg(['min', 'max', 'mean', 'std'])

#### **Method 1: Absolute Variation**

In [103]:
cost_variation_out['Range_Variation_Percentage'] = ((cost_variation_out['max'] - cost_variation_out['min']) / cost_variation_out['mean']) * 100

# Sort by highest variation (absolute method)
cost_variation_out_sorted = cost_variation_out.sort_values(by='Range_Variation_Percentage', ascending=False)
cost_variation_out_sorted['Range_Variation_Percentage'].head(5)

APC
0604 - Level 1 Hospital Clinic Visits             5732.175686
0013 - Level II Debridement & Destruction         1970.291482
0698 - Level II Eye Tests & Treatments            1642.126655
0692 - Level II Electronic Analysis of Devices    1274.364360
0078 - Level III Pulmonary Treatment              1180.307495
Name: Range_Variation_Percentage, dtype: float64

#### **Method 2: Relative Variation - Coefficient of Variation**

In [105]:
cost_variation_out['CV_Percentage'] = (cost_variation_out['std'] / cost_variation_out['mean']) * 100

# Sort by highest variation (relative method)
cost_variation_out_sorted = cost_variation_out.sort_values(by='CV_Percentage', ascending=False)
cost_variation_out_sorted['CV_Percentage'].head(5)

APC
0604 - Level 1 Hospital Clinic Visits             138.514954
0698 - Level II Eye Tests & Treatments            131.044419
0019 - Level I Excision/ Biopsy                   122.989724
0692 - Level II Electronic Analysis of Devices    112.816644
0012 - Level I Debridement & Destruction          105.008711
Name: CV_Percentage, dtype: float64

### Parts 1B & 1C: Highest-Cost Claims by Provider & Region

In [109]:
# Standardize column names (remove spaces)
Medicare_Provider_Charge_In.columns = Medicare_Provider_Charge_In.columns.str.strip().str.replace(' ', '_')
Medicare_Provider_Charge_Out.columns = Medicare_Provider_Charge_Out.columns.str.strip().str.replace(' ', '_')

In [113]:
Medicare_Provider_Charge_In.columns

Index(['DRG_Definition', 'Provider_Id', 'Provider_Name',
       'Provider_Street_Address', 'Provider_City', 'Provider_State',
       'Provider_Zip_Code', 'Hospital_Referral_Region_(HRR)_Description',
       'Total_Discharges', 'Average_Covered_Charges', 'Average_Total_Payments',
       'Average_Medicare_Payments'],
      dtype='object')

In [115]:
Medicare_Provider_Charge_Out.columns

Index(['APC', 'Provider_Id', 'Provider_Name', 'Provider_Street_Address',
       'Provider_City', 'Provider_State', 'Provider_Zip_Code',
       'Hospital_Referral_Region_(HRR)_Description', 'Outpatient_Services',
       'Average__Estimated_Submitted_Charges', 'Average_Total_Payments'],
      dtype='object')

In [121]:
# **1. Inpatient Providers: Total Charges per Provider**
inpatient_provider_charges = Medicare_Provider_Charge_In.groupby(['Provider_Id', 'Provider_Name'])['Average_Covered_Charges'].sum().reset_index()
inpatient_provider_charges.rename(columns={'Average_Covered_Charges': 'Total_Charges'}, inplace=True)

# **2. Outpatient Providers: Total Charges per Provider**
outpatient_provider_charges = Medicare_Provider_Charge_Out.groupby(['Provider_Id', 'Provider_Name'])['Average__Estimated_Submitted_Charges'].sum().reset_index()
outpatient_provider_charges.rename(columns={'Average__Estimated_Submitted_Charges': 'Total_Charges'}, inplace=True)

# **Sort both inpatient and outpatient providers by total charges**
top_inpatient_providers = inpatient_provider_charges.sort_values(by='Total_Charges', ascending=False).head(5)
top_outpatient_providers = outpatient_provider_charges.sort_values(by='Total_Charges', ascending=False).head(5)

# **Display Results**
print("Top 5 Highest-Cost Inpatient Providers:")
print(top_inpatient_providers)

print("\nTop 5 Highest-Cost Outpatient Providers:")
print(top_outpatient_providers)


Top 5 Highest-Cost Inpatient Providers:
      Provider_Id                  Provider_Name  Total_Charges
2531       390180  CROZER CHESTER MEDICAL CENTER   1.330169e+07
453         50625    CEDARS-SINAI MEDICAL CENTER   1.227909e+07
395         50441              STANFORD HOSPITAL   1.207722e+07
306         50195            WASHINGTON HOSPITAL   1.052081e+07
401         50464         DOCTORS MEDICAL CENTER   1.012871e+07

Top 5 Highest-Cost Outpatient Providers:
      Provider_Id                     Provider_Name  Total_Charges
414         50625       CEDARS-SINAI MEDICAL CENTER   99045.006822
655        100223  FORT WALTON BEACH MEDICAL CENTER   84200.262382
2360       390180     CROZER CHESTER MEDICAL CENTER   78892.060526
75          10131          CRESTWOOD MEDICAL CENTER   77658.405644
365         50441                 STANFORD HOSPITAL   75427.658276


## Method 2 : For INPATIENT

In [128]:
# **Step 1: Find Max Charge for Each Procedure (DRG)**
max_charges_per_DRG = Medicare_Provider_Charge_In.groupby('DRG_Definition')['Average_Covered_Charges'].max().reset_index()
max_charges_per_DRG.rename(columns={'Average_Covered_Charges': 'Max_Average_Covered_Charges'}, inplace=True)

# **Step 2: Merge with Original Data**
merged_inpatient = Medicare_Provider_Charge_In.merge(max_charges_per_DRG, on='DRG_Definition', how='left')

# **Step 3: Identify Providers with Max Charge**
merged_inpatient['Is_Max_Charge'] = merged_inpatient['Average_Covered_Charges'] == merged_inpatient['Max_Average_Covered_Charges']

# **Step 4: Count How Many Times Each Provider Had Max Charge**
provider_max_count = merged_inpatient.groupby('Provider_Name')['Is_Max_Charge'].sum().reset_index()
provider_max_count.rename(columns={'Is_Max_Charge': 'Max_Charge_Count'}, inplace=True)

# **Step 5: Count How Many Times Each Region Had Max Charge**
region_max_count = merged_inpatient.groupby('Hospital_Referral_Region_(HRR)_Description')['Is_Max_Charge'].sum().reset_index()
region_max_count.rename(columns={'Is_Max_Charge': 'Max_Charge_Count'}, inplace=True)

# **Step 6: Get Top 3 Providers and Top 5 Regions**
top_3_providers = provider_max_count.nlargest(5, 'Max_Charge_Count')
top_5_regions = region_max_count.nlargest(5, 'Max_Charge_Count')

# **Display Results**
print("Top 5 Providers with Highest Charges:")
print(top_3_providers)

print("\nTop 5 Regions with Highest Charges:")
print(top_5_regions)


Top 5 Providers with Highest Charges:
                      Provider_Name  Max_Charge_Count
202         BAYONNE HOSPITAL CENTER                23
590   CROZER CHESTER MEDICAL CENTER                11
2730              STANFORD HOSPITAL                10
982   HAHNEMANN UNIVERSITY HOSPITAL                 7
661          DOCTORS MEDICAL CENTER                 5

Top 5 Regions with Highest Charges:
    Hospital_Referral_Region_(HRR)_Description  Max_Charge_Count
191                                NJ - Newark                25
232                          PA - Philadelphia                23
34                       CA - San Mateo County                13
19                    CA - Contra Costa County                 6
16                         CA - Alameda County                 5


## For OUTPATIENT

In [131]:
# **Step 1: Find Max Charge for Each Procedure (APC)**
max_charges_per_APC = Medicare_Provider_Charge_Out.groupby('APC')['Average__Estimated_Submitted_Charges'].max().reset_index()
max_charges_per_APC.rename(columns={'Average__Estimated_Submitted_Charges': 'Max_Average_Submitted_Charges'}, inplace=True)

# **Step 2: Merge with Original Data**
merged_outpatient = Medicare_Provider_Charge_Out.merge(max_charges_per_APC, on='APC', how='left')

# **Step 3: Identify Providers with Max Charge**
merged_outpatient['Is_Max_Charge'] = merged_outpatient['Average__Estimated_Submitted_Charges'] == merged_outpatient['Max_Average_Submitted_Charges']

# **Step 4: Count How Many Times Each Provider Had Max Charge**
provider_max_count_out = merged_outpatient.groupby('Provider_Name')['Is_Max_Charge'].sum().reset_index()
provider_max_count_out.rename(columns={'Is_Max_Charge': 'Max_Charge_Count'}, inplace=True)

# **Step 5: Count How Many Times Each Region Had Max Charge**
region_max_count_out = merged_outpatient.groupby('Hospital_Referral_Region_(HRR)_Description')['Is_Max_Charge'].sum().reset_index()
region_max_count_out.rename(columns={'Is_Max_Charge': 'Max_Charge_Count'}, inplace=True)

# **Step 6: Get Top 5 Providers and Top 5 Regions**
top_3_providers_out = provider_max_count_out.nlargest(5, 'Max_Charge_Count')
top_5_regions_out = region_max_count_out.nlargest(5, 'Max_Charge_Count')

# **Display Results**
print("Top 5 Outpatient Providers with Highest Charges:")
print(top_3_providers_out)

print("\nTop 5 Outpatient Regions with Highest Charges:")
print(top_5_regions_out)


Top 5 Outpatient Providers with Highest Charges:
                                Provider_Name  Max_Charge_Count
327  CAPITAL HEALTH MEDICAL CENTER - HOPEWELL                 2
484                         COLUMBIA HOSPITAL                 2
799          FORT WALTON BEACH MEDICAL CENTER                 2
228              BEVERLY HOSPITAL CORPORATION                 1
372         CENTINELA HOSPITAL MEDICAL CENTER                 1

Top 5 Outpatient Regions with Highest Charges:
    Hospital_Referral_Region_(HRR)_Description  Max_Charge_Count
230                          PA - Philadelphia                 5
21                            CA - Los Angeles                 3
24                          CA - Orange County                 2
54                        FL - Fort Lauderdale                 2
65                              FL - Pensacola                 2


### Part 1D: Highest Number of Procedures and Largest Differences between Claims and Reimbursements

### INPATIENT

In [12]:
# **Step 1: Calculate the Difference Between Claims and Reimbursements**
Medicare_Provider_Charge_In['Claim_Difference'] = Medicare_Provider_Charge_In['Average_Covered_Charges'] - Medicare_Provider_Charge_In['Average_Total_Payments']

# **Step 2: Calculate the Mean and Max Claim Difference by DRG Definition**
mean_claim_diff_by_DRG = Medicare_Provider_Charge_In.groupby('DRG_Definition')['Claim_Difference'].mean().reset_index()
max_claim_diff_by_DRG = Medicare_Provider_Charge_In.groupby('DRG_Definition')['Claim_Difference'].max().reset_index()

# **Step 3: Merge the Mean and Max Values with the Original Data**
merged_inpatient = Medicare_Provider_Charge_In.merge(mean_claim_diff_by_DRG, on='DRG_Definition', how='left', suffixes=('', '_Mean'))
merged_inpatient = merged_inpatient.merge(max_claim_diff_by_DRG, on='DRG_Definition', how='left', suffixes=('', '_Max'))

# **Step 4: Identify Providers and Regions with the Largest Differences**
merged_inpatient['Is_Max_Claim_Difference'] = merged_inpatient['Claim_Difference'] == merged_inpatient['Claim_Difference_Max']
merged_inpatient['Is_Max_Claim_Difference_Mean'] = merged_inpatient['Claim_Difference'] == merged_inpatient['Claim_Difference_Mean']

# **Step 5: Count How Many Times Providers Have the Max Claim Difference**
provider_max_diff_count = merged_inpatient.groupby('Provider_Name')['Is_Max_Claim_Difference'].sum().reset_index()
provider_max_diff_count.rename(columns={'Is_Max_Claim_Difference': 'Max_Claim_Diff_Count'}, inplace=True)

# **Step 6: Get Top Providers with Max Differences**
top_3_providers_max_diff = provider_max_diff_count.nlargest(3, 'Max_Claim_Diff_Count')

# **Display Results**
print("Top 3 Providers with Highest Claim Differences:")
print(top_3_providers_max_diff)


Top 3 Providers with Highest Claim Differences:
                     Provider_Name  Max_Claim_Diff_Count
202        BAYONNE HOSPITAL CENTER                    27
590  CROZER CHESTER MEDICAL CENTER                    12
982  HAHNEMANN UNIVERSITY HOSPITAL                     8


### OUTPATIENT

In [145]:
# **Step 1: Calculate the Difference Between Claims and Reimbursements**
Medicare_Provider_Charge_Out['Claim_Difference'] = Medicare_Provider_Charge_Out['Average__Estimated_Submitted_Charges'] - Medicare_Provider_Charge_Out['Average_Total_Payments']

# **Step 2: Calculate the Mean and Max Claim Difference by APC**
mean_claim_diff_by_APC = Medicare_Provider_Charge_Out.groupby('APC')['Claim_Difference'].mean().reset_index()
max_claim_diff_by_APC = Medicare_Provider_Charge_Out.groupby('APC')['Claim_Difference'].max().reset_index()

# **Step 3: Merge the Mean and Max Values with the Original Data**
merged_outpatient = Medicare_Provider_Charge_Out.merge(mean_claim_diff_by_APC, on='APC', how='left', suffixes=('', '_Mean'))
merged_outpatient = merged_outpatient.merge(max_claim_diff_by_APC, on='APC', how='left', suffixes=('', '_Max'))

# **Step 4: Identify Providers and Regions with the Largest Differences**
merged_outpatient['Is_Max_Claim_Difference'] = merged_outpatient['Claim_Difference'] == merged_outpatient['Claim_Difference_Max']
merged_outpatient['Is_Max_Claim_Difference_Mean'] = merged_outpatient['Claim_Difference'] == merged_outpatient['Claim_Difference_Mean']

# **Step 5: Count How Many Times Providers Have the Max Claim Difference**
provider_max_diff_count_out = merged_outpatient.groupby('Provider_Name')['Is_Max_Claim_Difference'].sum().reset_index()
provider_max_diff_count_out.rename(columns={'Is_Max_Claim_Difference': 'Max_Claim_Diff_Count'}, inplace=True)

# **Step 6: Get Top Providers with Max Differences**
top_3_providers_max_diff_out = provider_max_diff_count_out.nlargest(3, 'Max_Claim_Diff_Count')

# **Display Results**
print("Top 3 Providers with Highest Claim Differences (Outpatient):")
print(top_3_providers_max_diff_out)


Top 3 Providers with Highest Claim Differences (Outpatient):
                                Provider_Name  Max_Claim_Diff_Count
327  CAPITAL HEALTH MEDICAL CENTER - HOPEWELL                     2
484                         COLUMBIA HOSPITAL                     2
799          FORT WALTON BEACH MEDICAL CENTER                     2
